# 1. 問題の分析

## 競技プログラミング視点での分析

**実行速度を最優先とした場合のアプローチ:**
- プロトタイプチェーン探索は O(d) で済む（d = チェーン深度、通常3-10程度）
- `isPrototypeOf()` はV8のネイティブ実装で最速
- 早期リターンで不要な処理を徹底的に削減
- プリミティブの `Object()` 変換は避けられないが、1回のみに抑制

**メモリ使用量の最小化方針:**
- 新規オブジェクト生成は `Object(primitive)` のみ（プリミティブ時のみ）
- プロトタイプチェーン探索は既存参照のみ使用
- スタックフレーム最小化（while使用、再帰回避）

## 業務開発視点での分析

**型安全性・保守性・可読性を重視した場合のアプローチ:**
- TypeScriptの型システムで `unknown` 型を活用
- ジェネリクスで型安全な汎用実装
- エッジケース（null, undefined, プリミティブ）を型レベルで表現
- JSDocと型定義の併用で意図を明確化

**エラーハンドリング・型安全性の考慮:**
- 型ガードで実行時の型安全性を確保
- `null`/`undefined` は型レベルで除外
- 不正な `classFunction` も型で制約（`Function` 型）
- 例外を投げず、`boolean` で結果を返す（関数型的アプローチ）

## TypeScript特有の考慮点

**型推論とコンパイル時最適化:**
- ジェネリクスで型パラメータを保持しつつ実装
- `unknown` 型から安全に型変換
- 戻り値の型ガードで型推論を活用

**ジェネリクスの効果的な活用:**
```typescript
// クラスコンストラクタの型を正確に表現
type Constructor<T = any> = new (...args: any[]) => T;
type AnyFunction = (...args: any[]) => any;
```

**型ガードとnull安全性:**
- `obj == null` で `null` と `undefined` を同時チェック
- Optional Chaining (`?.`) でnullish値の安全なアクセス
- Nullish Coalescing (`??`) でデフォルト値設定

---

# 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---------|----------|----------|------------|---------|--------|-----|
| isPrototypeOf使用 | O(d) | O(1) | 低 | 高 | 最高 | V8ネイティブ実装で最速 |
| __proto__直接走査 | O(d) | O(1) | 低 | 中 | 中 | 非標準、型定義が曖昧 |
| Object.getPrototypeOf | O(d) | O(1) | 低 | 高 | 高 | 標準だが関数呼び出しコスト |
| instanceof演算子 | O(d) | O(1) | 最低 | 高 | 最高 | プリミティブで失敗（要件不適合） |

**注:** d（depth）は通常3-10。TypeScriptでは型安全性と実行速度の両立が重要。

---

# 3. 選択したアルゴリズムと理由

**選択したアプローチ:** `isPrototypeOf()` を活用したプロトタイプチェーン検証

**理由:**

1. **計算量的な優位性:**
   - O(d) で最小、V8のC++実装で最速
   - while ループより組み込みメソッドの方が高速

2. **TypeScript環境での型安全性:**
   - `unknown` 型からの安全な型変換
   - ジェネリクスで汎用的な実装
   - Optional Chaining で null 安全性確保

3. **保守性・可読性の観点:**
   - `isPrototypeOf()` は意図が明確
   - 型定義でドキュメント化
   - エッジケースが型で表現される

**TypeScript特有の最適化ポイント:**

1. **コンパイル時の型チェックによるエラー防止:**
   - `classFunction` を `Function` 型に制約
   - 戻り値は必ず `boolean`（型安全）

2. **ジェネリクスによる再利用性:**
   - `Constructor<T>` 型で任意のクラスに対応
   - 型パラメータで型情報を保持

3. **型推論による開発効率向上:**
   - 引数の型から戻り値が推論される
   - IntelliSense による開発支援

---

# 4. 実装コード

Analyze Complexity
Runtime 68 ms
Beats 72.43%
Memory 63.54 MB
Beats 57.48%

```typescript
/**
 * クラスコンストラクタの型定義
 * @template T - コンストラクタが生成するインスタンスの型
 */
type Constructor<T = any> = new (...args: any[]) => T;

/**
 * 任意の関数型（クラスコンストラクタを含む）
 */
type AnyFunction = (...args: any[]) => any;

/**
 * 値が指定されたクラスまたはスーパークラスのインスタンスかチェック
 * 
 * JavaScriptの instanceof とは異なり、プリミティブ値も対応:
 * - 5 は Number のインスタンスとみなす
 * - "hello" は String のインスタンスとみなす
 * - true は Boolean のインスタンスとみなす
 * 
 * @param obj - チェック対象の値（任意の型、null/undefined含む）
 * @param classFunction - クラスコンストラクタ（null/undefined含む任意の値）
 * @returns obj が classFunction（またはそのスーパークラス）のインスタンスなら true
 * 
 * @example
 * ```typescript
 * checkIfInstanceOf(new Date(), Date) // true
 * checkIfInstanceOf(5, Number) // true
 * checkIfInstanceOf(5, String) // false
 * checkIfInstanceOf(null, Object) // false
 * 
 * class Animal {}
 * class Dog extends Animal {}
 * checkIfInstanceOf(new Dog(), Animal) // true
 * ```
 * 
 * @complexity 
 * - Time: O(d) - d はプロトタイプチェーンの深度（通常3-10）
 * - Space: O(1) - 固定サイズの変数のみ使用
 */
function checkIfInstanceOf(
    obj: unknown,
    classFunction: unknown
): boolean {
    // 型ガード: null/undefined の早期リターン
    if (obj == null) {
        return false;
    }
    
    // 型ガード: classFunction が関数でない場合
    if (typeof classFunction !== 'function') {
        return false;
    }
    
    // この時点で classFunction は Function 型
    // TypeScript の型推論により、以降は安全にアクセス可能
    
    // プリミティブ値の処理
    // typeof による型判定（型ガード）
    const objType = typeof obj;
    
    if (objType !== 'object' && objType !== 'function') {
        // プリミティブの場合: Object() でラッパーオブジェクト化
        // 例: Object(5) → Number {5}
        obj = Object(obj);
    }
    
    // この時点で obj は object | function 型
    
    // Optional Chaining で null 安全性を確保
    // classFunction.prototype が undefined の場合（Arrow関数等）は false
    // isPrototypeOf() は V8 の最適化されたネイティブ実装
    const prototype = (classFunction as AnyFunction).prototype;
    
    if (prototype == null) {
        return false;
    }
    
    // isPrototypeOf による型安全なチェック
    // obj as object はこの時点で安全（上記の型ガードにより保証）
    return prototype.isPrototypeOf(obj as object);
}

// LeetCode提出用エクスポート
export { checkIfInstanceOf };

/**
 * LeetCode フォーマット（var 宣言）
 */

// Analyze Complexity
// Runtime 70 ms
// Beats 64.02%
// Memory 63.97 MB
// Beats 30.37%

var checkIfInstanceOf = function(obj: unknown, classFunction: unknown): boolean {
    if (obj == null || typeof classFunction !== 'function') {
        return false;
    }
    
    if (typeof obj !== 'object' && typeof obj !== 'function') {
        obj = Object(obj);
    }
    
    return (classFunction as AnyFunction).prototype?.isPrototypeOf(obj as object) ?? false;
};
```

---

# 5. TypeScript固有の最適化観点

## 型安全性の活用

### 1. コンパイル時エラー防止

```typescript
// ❌ JavaScriptでは実行時エラー
// checkIfInstanceOf(5, "not a function")

// ✅ TypeScriptでは引数型で制約可能（ただし問題要件上 unknown を受け入れる）
// 実行時の型ガードで安全性を確保
if (typeof classFunction !== 'function') {
    return false; // 例外を投げずに安全に処理
}
```

### 2. null/undefined安全性の確保

```typescript
// Nullish Coalescing と Optional Chaining の活用
return classFunction.prototype?.isPrototypeOf(obj) ?? false;

// これは以下と等価だが、より簡潔で型安全
if (classFunction.prototype == null) {
    return false;
}
return classFunction.prototype.isPrototypeOf(obj);
```

### 3. ジェネリクスによる再利用性

```typescript
// 将来的な拡張を考慮した型定義
type Constructor<T = any> = new (...args: any[]) => T;

// 使用例（より厳密な型チェックが必要な場合）
function strictCheckIfInstanceOf<T>(
    obj: unknown,
    classFunction: Constructor<T>
): obj is T {
    return checkIfInstanceOf(obj, classFunction);
}

// 型ガードとして使用可能
if (strictCheckIfInstanceOf(value, Date)) {
    // この時点で value は Date 型として扱われる
    console.log(value.getFullYear());
}
```

## コンパイル時最適化

### 1. 型推論の活用

```typescript
// 明示的な型注釈は最小限に
const objType = typeof obj; // string 型と自動推論
if (objType !== 'object' && objType !== 'function') {
    // TypeScript が自動的に型を絞り込む
}
```

### 2. readonly修飾子（将来的な拡張）

```typescript
// イミュータブルなデータ構造（副作用防止）
interface CheckOptions {
    readonly strict?: boolean;
    readonly allowPrimitives?: boolean;
}
```

### 3. const assertion

```typescript
// リテラル型の活用
const PRIMITIVE_TYPES = ['number', 'string', 'boolean', 'symbol', 'bigint'] as const;
type PrimitiveType = typeof PRIMITIVE_TYPES[number];
```

## 開発効率と保守性

### IntelliSenseによる開発支援

```typescript
/**
 * JSDoc により、VSCode等でホバー時に詳細な説明が表示される
 * 引数の型、戻り値、使用例、計算量まで確認可能
 */
```

### リファクタリング安全性

```typescript
// 型定義があるため、関数名変更時も自動追跡
// classFunction のプロパティアクセスもコンパイラがチェック
```

### チーム開発での型情報共有

```typescript
// Constructor<T> 型により、他の開発者も意図を理解しやすい
// unknown 型の使用により、任意の値を受け入れることが明示的
```

---

## 期待されるパフォーマンス

| 項目 | 予想値 |
|-----|-------|
| Runtime | **50-65ms (70-85%)** |
| Memory | **62MB (95%+)** |
| Test Pass | **311/311 (100%)** |

### 最適化の根拠

1. **`isPrototypeOf()` の活用:** V8のネイティブ実装で最速
2. **早期リターン:** 不要な処理を徹底排除
3. **型ガードの最適配置:** コンパイル時と実行時の両方で安全性確保
4. **プリミティブ処理の最小化:** `Object()` 呼び出しは1回のみ